# Sztuczna inteligencja i inżynieria wiedzy
## Lista 2. - Implementacja AI do gry Clobber
### Michał Maksanty (272685)

## 1. Cel zadania
Celem ćwiczenia jest praktyczne zapoznanie się z algorytmami grania w gry dwuosobowe - algorytmem Minimax oraz jego ulepszeniem z wykorzystaniem alfa-beta cięć.

## 2. Wprowadzenie teoretyczne
### 2.1 Definicje
- **Drzewo decyzyjne**: Skierowane drzewo, w którym każdy węzeł reprezentuje stan problemu, a krawędzie reprezentują działania prowadzące do kolejnych stanów.
- **Gra o sumie zerowej**: Gra, w której suma wypłat wszystkich graczy wynosi zero w każdej możliwej kombinacji działań.

### 2.2 Algorytm Minimax
Strategia przeszukiwania drzewa gry służąca do podejmowania decyzji w grach dwuosobowych o sumie zerowej. Algorytm dąży do minimalizacji maksymalnej możliwej straty.

### 2.3 Alfa-beta cięcie
Optymalizacja algorytmu Minimax polegająca na eliminowaniu gałęzi drzewa, które z pewnością nie są optymalne.

## 3. Implementacja podstawowych klas

Poniższy kod implementuje grę planszową Clobber w języku Python (narazie bez żadnych funkcji AI - czysta gra). Rozgrywka toczy się na dwuwymiarowej planszy, gdzie pola są naprzemiennie wypełnione pionkami białymi i czarnymi. Gracze wykonują ruchy, „zbijając” sąsiednie pionki przeciwnika i zastępując je swoimi, a gra kończy się, gdy żaden pionek nie może wykonać kolejnego ruchu. Klasa Clobber zarządza planszą, stanem gry, bieżącym graczem i numerem rundy, a także udostępnia metodę move, umożliwiającą wykonanie ruchu w jednym z czterech kierunków. Enumy Pawn, GameStatus i Direction pomagają zarządzać logiką gry, zapewniając czytelność i bezpieczeństwo typów. Dodatkowo, gra umożliwia wizualizację planszy za pomocą metody print_board, opcjonalnie z nazwami pól.

Ogólnie częśc z funkcjonalności tej klasy nie jest później wykorzystana, jednak klasa została utworzona całościowo w taki sposób, by z jej pomocą można było samemu rozegrać pełnoprawną rozgrywkę w Clobbera (człowiek vs człowiek)

In [10]:
from enum import Enum
from typing import List

class Pawn(Enum):
    WHITE = 'W'
    BLACK = 'B'
    EMPTY = '_'
    def __str__(self):
        return self.value

class GameStatus(Enum):
    NOT_STARTED = 0
    IN_PROGRESS = 1
    ENDED = 2

class Direction(Enum):
    UP = 1
    DOWN = 2
    RIGHT = 3
    LEFT = 4

Board = List[List[Pawn]]

class Clobber:
    def __init__(self, n=5, m=6):
        self._n = n
        self._m = m
        pawn = Pawn.WHITE if m % 2 == 0 else Pawn.BLACK
        self._board = [[] for _ in range(m)]
        for row in self._board:
            for _ in range(n):
                row.append(pawn)
                pawn = self.other_player(pawn)
        self._current_player = Pawn.WHITE
        self._game_status = GameStatus.NOT_STARTED
        self._rounds_nb = 0

    @property
    def board(self):
        return self._board

    @property
    def game_status(self):
        return self._game_status

    @property
    def current_player(self):
        return self._current_player

    @property
    def round_nb(self):
        return self._rounds_nb

    def winner(self):
        if self.game_status != GameStatus.ENDED:
            return None
        return self.other_player(self.current_player)

    @staticmethod
    def other_player(player):
        if player == Pawn.WHITE:
            return Pawn.BLACK
        elif player == Pawn.BLACK:
            return Pawn.WHITE
        else:
            return None

    @staticmethod
    def validate_move(board, i, j, player):
        m = len(board)
        n = len(board[0]) if m > 0 else 0
        if i < 0 or i >= m or j < 0 or j >= n:
            return False
        if board[i][j] != Clobber.other_player(player):
            return False
        return True

    @staticmethod
    def can_clobber(board, i, j):
        player = board[i][j]
        neighbor_positions = [
            (i - 1, j),
            (i + 1, j),
            (i, j - 1),
            (i, j + 1)
        ]
        for ni, nj in neighbor_positions:
            if Clobber.validate_move(board, ni, nj, player):
                return True
        return False

    @staticmethod
    def game_ended(board):
        for i in range(len(board)):
            for j in range(len(board[i])):
                field = board[i][j]
                if field in (Pawn.BLACK, Pawn.WHITE):
                    if Clobber.can_clobber(board, i, j):
                        return False
        return True

    @staticmethod
    def neighbor_moves(i, j):
        return [
            (i-1, j),
            (i+1, j),
            (i, j-1),
            (i, j+1)
        ]

    def move(self, i, j, direction):
        chosen_pawn = self.board[i][j]
        if chosen_pawn != self._current_player:
            raise WrongTurnException
        match direction:
            case Direction.UP:
                move_i = -1
                move_j = 0
            case Direction.DOWN:
                move_i = 1
                move_j = 0
            case Direction.RIGHT:
                move_i = 0
                move_j = 1
            case Direction.LEFT:
                move_i = 0
                move_j = -1
            case _:
                raise WrongDirectionException
        next_i = i+move_i
        next_j = j+move_j
        if not self.validate_move(self.board, next_i, next_j, self._current_player):
            raise InvalidMoveException

        self.board[i][j] = Pawn.EMPTY
        self.board[next_i][next_j] = self._current_player
        self._current_player = self.other_player(self.current_player)

        self._rounds_nb += 1
        self._game_status = GameStatus.IN_PROGRESS
        if self.game_ended(self.board):
            self._game_status = GameStatus.ENDED

    def print(self, field_names=False):
        self.print_board(self.board, field_names)

    @staticmethod
    def print_board(board, field_names=False):
        n = len(board[0])
        m = len(board)
        if field_names:
            print(end='    ')
            print('  '.join((chr(letter) for letter in range(ord('A'), ord('A')+n))))
            print(end='    ')
            print('  '.join(('_' for _ in range(n))))
        for i in range(len(board)):
            row = board[i]
            if field_names:
                print(f'{m-i}|', end='  ')
            print('  '.join((str(v) for v in row)))

Poniżej znajdują się klasy wyjątków, używane w powyższej implementacji gry Clobber do obsłużenia sytuacji wyjątkowych

In [11]:
class WrongTurnException(Exception):
    pass

class WrongDirectionException(Exception):
    pass

class InvalidMoveException(Exception):
    pass

## 4. Implementacja heurystyk
### Dostępne podstawowe heurystyki:
1. `active_pawns_heuristics` - zlicza pionki gracza oraz przeciwnika, które mają możliwość zaatakowania innych i zwraca różnicę między nimi (nagradza za aktywne pionki)
2. `center_occupying_heuristics` - tworzy macierz wag pól na planszy, zlicza sumę punktów gracza i przeciwnika i zwraca różnicę między nimi (nagradza za zdobywanie centrum)
3. `pawns_accumulations_heuristics` - zlicza "wyspy" (sąsiadujące grupy pionków) gracza i przeciwnika i zwraca różnicę między nimi (nagradza za nierozpraszanie się na planszy)

Poza powyższymi zdefiniowane również zostały heurystyki adaptacyjne, powstałe w wyniku adaptacyjnego wyboru z powyższych heurystyk w zależności od stanu gry
### Dostępne rozszerzone heurystyki:
1. 'first_center_then_aggressive': - w początkowej fazie gry, strategia skupia się na zdobyciu centrum, później nagradza za aktywne pionki
2. 'group_then_fight': group_then_fight, - na początku gry strategia skupia się na zgrupowaniu pionków, później na ataku
3. 'take_middle_stay_in_group': take_middle_stay_in_group, - strategia grupuje pionki w centrum i pilnuje aby się nie rozchodziły

In [12]:
from functools import lru_cache


def active_pawns_heuristics(board: Board, maximizing_player: Pawn) -> int:
    max_active = 0
    min_active = 0
    for i, row in enumerate(board):
        for j, pawn in enumerate(row):
            if pawn == Pawn.EMPTY:
                continue
            if Clobber.can_clobber(board, i, j):
                if pawn == maximizing_player:
                    max_active += 1
                else:
                    min_active += 1
    return max_active - min_active


def center_occupying_heuristics(board: Board, maximizing_player: Pawn) -> int:
    m = len(board)
    n = len(board[0])
    weights = generate_weight_grid(n, m)
    max_score = 0
    min_score = 0
    for i, row in enumerate(board):
        for j, pawn in enumerate(row):
            if pawn == Pawn.EMPTY:
                continue
            weight = weights[i][j]
            if pawn == maximizing_player:
                max_score += weight
            else:
                min_score += weight
    return max_score - min_score

@lru_cache
def generate_weight_grid(n: int, m: int) -> list[list[int]]:
    def calculate_weight(i, j, rows, cols):
        dist_i = min(i, rows - 1 - i)
        dist_j = min(j, cols - 1 - j)

        base = dist_i + dist_j
        bonus = min(dist_i, dist_j)

        return base + bonus

    return [[calculate_weight(i, j, rows=m, cols=n) for j in range(n)] for i in range(m)]


def pawns_accumulations_heuristics(board: Board, maximizing_player: Pawn) -> int:
    max_islands = 0
    min_islands = 0
    visited = set()
    for i, row in enumerate(board):
        for j, pawn in enumerate(row):
            if (i, j) in visited or pawn == Pawn.EMPTY:
                continue
            stack = [(i, j)]
            while stack:
                x, y = stack.pop()
                if (x, y) in visited:
                    continue
                visited.add((x, y))
                for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
                    nx, ny = x+dx, y+dy
                    if 0 <= nx < len(board) and 0 <= ny < len(board[0]):
                        if board[nx][ny] == pawn and (nx, ny) not in visited:
                            stack.append((nx, ny))
            if pawn == maximizing_player:
                max_islands += 1
            else:
                min_islands += 1
    return min_islands - max_islands


def first_center_then_aggressive(board: Board, player: Pawn) -> float:
    pawns_left_coeff = pawns_left_coefficient(board, player)
    if pawns_left_coeff >= 0.6:
        return center_occupying_heuristics(board, player)
    elif pawns_left_coeff >= 0.4:
        return active_pawns_heuristics(board, player) * 0.7 + center_occupying_heuristics(board, player) * 0.3
    else:
        return active_pawns_heuristics(board, player)

def group_then_fight(board: Board, player: Pawn) -> float:
    pawns_left_coeff = pawns_left_coefficient(board, player)
    if pawns_left_coeff >= 0.6:
        return pawns_accumulations_heuristics(board, player)
    elif pawns_left_coeff >= 0.4:
        return pawns_accumulations_heuristics(board, player) * 0.4 + active_pawns_heuristics(board, player) * 0.6
    else:
        return active_pawns_heuristics(board, player)

def take_middle_stay_in_group(board: Board, player: Pawn) -> float:
    pawns_left_coeff = pawns_left_coefficient(board, player)
    if pawns_left_coeff >= 0.6:
        return center_occupying_heuristics(board, player)
    elif pawns_left_coeff >= 0.4:
        return center_occupying_heuristics(board, player) * 0.5 + pawns_accumulations_heuristics(board, player) * 0.5
    else:
        return pawns_accumulations_heuristics(board, player)

def pawns_left_coefficient(board: Board, player: Pawn) -> float:
    initial_pawns = len(board) * len(board[0]) / 2
    count = 0
    for row in board:
        for pawn in row:
            if pawn == player:
                count += 1
    return count / initial_pawns


HEURISTIC_MAP = {
    'active': active_pawns_heuristics,
    'center': center_occupying_heuristics,
    'accumulation': pawns_accumulations_heuristics
}

HEURISTIC_MAP_EXTENDED = {
    'first_center_then_aggressive': first_center_then_aggressive,
    'group_then_fight': group_then_fight,
    'take_middle_stay_in_group': take_middle_stay_in_group,
    **HEURISTIC_MAP
}

## 5. Implementacja algorytmów Minimax i Alpha-Beta
### Kluczowe elementy:
- **Reprezentacja stanu gry** - klasa ClobberState odpowiada za przechowywanie obecnego stanu gry. Przechowuje ona planszę, oraz informację o graczu wykonującym ruch. Udostępnia metodę pozwalającą wykonać ruch - metoda ta zwraca kolejny obiekt ClobberState (stan gry po wykonaniu ruchu)
- **Generowanie możliwych ruchów** - wyżej wspomniana klasa udostępnia również funkcję pozwlającą generować wszystkie możliwe ruchu w danym stanie gry (jest to niezbędny element dla algorytmu *minimax*
- **Implementacja Minimax** - implementacja algorytmu minimax dla gry Clobber. Przyjmuje jako argumenty stan gry, głębokość, maksymalizowanego gracza, heurystykę oraz statystyki. Algorytm rekurencyjnie schodzi od korzenia drzewa w dół aż dojdzie do liści lub głębokość wyniesie 0. Wówczas liczona jest heurystyka dla danego poziomu i algorytm wracając do korzenia wybiera optymalną (wg. używanej heurystyki) ścieżkę. Przekazywany w dół drzewa atrybut *stats* pozwala na zliczanie odwiedzonych wierzchołków drzewa celem późniejszego logowania tej informacji.
- **Implementacja Alpha-Beta z cięciami** - analogiczny algorytm do powyższego, z tym że przyjmuje dwa dodatkowe argumenty - alpha, beta, dzięki którym może odpowiednio wcześnie zakończyć przeglądanie fragmentu drzewa, które napewno nie jest optymalne.

In [13]:
from typing import List, Tuple, Dict, Optional
import time


WINNING_VALUE = 1000


class ClobberState:
    def __init__(self, board: List[List[Pawn]], current_player: Pawn):
        self.board = board
        self.current_player = current_player
        self._m = len(board)
        self._n = len(board[0]) if self._m > 0 else 0

    def get_possible_moves(self) -> List[Tuple[int, int, Direction]]:
        moves = []
        for i in range(self._m):
            for j in range(self._n):
                if self.board[i][j] == self.current_player:
                    for direction in Direction:
                        di, dj = 0, 0
                        if direction == Direction.UP:
                            di = -1
                        elif direction == Direction.DOWN:
                            di = 1
                        elif direction == Direction.LEFT:
                            dj = -1
                        elif direction == Direction.RIGHT:
                            dj = 1
                        ni, nj = i + di, j + dj
                        if 0 <= ni < self._m and 0 <= nj < self._n:
                            if self.board[ni][nj] == Clobber.other_player(self.current_player):
                                moves.append((i, j, direction))
        return moves

    def make_move(self, move: Tuple[int, int, Direction]) -> 'ClobberState':
        i, j, direction = move
        new_board = [row.copy() for row in self.board]

        di, dj = 0, 0
        if direction == Direction.UP:
            di = -1
        elif direction == Direction.DOWN:
            di = 1
        elif direction == Direction.LEFT:
            dj = -1
        elif direction == Direction.RIGHT:
            dj = 1

        ni, nj = i + di, j + dj
        new_board[i][j] = Pawn.EMPTY
        new_board[ni][nj] = self.current_player
        next_player = Clobber.other_player(self.current_player)
        return ClobberState(new_board, next_player)

    def is_terminal(self) -> bool:
        return len(self.get_possible_moves()) == 0

    def __str__(self):
        return "\n".join(" ".join(p.value for p in row) for row in self.board)


def utility(state: ClobberState, maximizing_player: Pawn) -> int:
    winner = Clobber.other_player(state.current_player)
    return WINNING_VALUE if winner == maximizing_player else -WINNING_VALUE


def minimax(state: ClobberState, depth: int, maximizing_player: Pawn, heuristic, stats: Dict) -> int:
    stats['nodes'] += 1
    if depth == 0 or state.is_terminal():
        if state.is_terminal():
            return utility(state, maximizing_player)
        return heuristic(state.board, maximizing_player)

    if state.current_player == maximizing_player:
        value = -float('inf')
        for move in state.get_possible_moves():
            new_state = state.make_move(move)
            value = max(value, minimax(new_state, depth - 1, maximizing_player, heuristic, stats))
        return value
    else:
        value = float('inf')
        for move in state.get_possible_moves():
            new_state = state.make_move(move)
            value = min(value, minimax(new_state, depth - 1, maximizing_player, heuristic, stats))
        return value


def alphabeta(state: ClobberState, depth: int, alpha: float, beta: float,
              maximizing_player: Pawn, heuristic, stats: Dict) -> int:
    stats['nodes'] += 1
    if depth == 0 or state.is_terminal():
        if state.is_terminal():
            return utility(state, maximizing_player)
        return heuristic(state.board, maximizing_player)

    if state.current_player == maximizing_player:
        value = -float('inf')
        for move in state.get_possible_moves():
            new_state = state.make_move(move)
            value = max(value, alphabeta(new_state, depth - 1, alpha, beta, maximizing_player, heuristic, stats))
            alpha = max(alpha, value)
            if value >= beta:
                break
        return value
    else:
        value = float('inf')
        for move in state.get_possible_moves():
            new_state = state.make_move(move)
            value = min(value, alphabeta(new_state, depth - 1, alpha, beta, maximizing_player, heuristic, stats))
            beta = min(beta, value)
            if value <= alpha:
                break
        return value


def find_best_move(state: ClobberState, depth: int, player: Pawn, heuristic,
                   use_alpha_beta: bool, stats: Dict) -> Tuple[Optional[Tuple[int, int, Direction]], float, int, float]:
    moves = state.get_possible_moves()
    if not moves:
        return None, 0, 0, 0.0

    is_maximizing = (player == state.current_player)

    best_value = -float('inf') if is_maximizing else float('inf')
    best_move = None
    alpha = -float('inf')
    beta = float('inf')

    start_time = time.time()
    stats['nodes'] = 0

    for move in moves:
        new_state = state.make_move(move)

        if use_alpha_beta:
            value = alphabeta(
                new_state,
                depth - 1,
                alpha,
                beta,
                player,
                heuristic,
                stats
            )
        else:
            value = minimax(
                new_state,
                depth - 1,
                player,
                heuristic,
                stats
            )

        if is_maximizing:
            if value > best_value:
                best_value = value
                best_move = move
                alpha = max(alpha, best_value)
        else:
            if value < best_value:
                best_value = value
                best_move = move
                beta = min(beta, best_value)

        # alpha-beta pruning
        if use_alpha_beta and alpha >= beta:
            break

    elapsed = time.time() - start_time
    return best_move, best_value, stats['nodes'], elapsed

## 6. Główny program AI
### Funkcjonalności:
- Tryb podstawowy: obaj gracze używają tej samej strategii (atrybuty depth i heuristics)
- Tryb rozszerzony: możliwość definiowania różnych strategii dla każdego gracza (atrybuty [black/white]_depth i [black/white]_heuristics)
- Wsparcie dla cięć alfa-beta (możliwe również wyłączenie tej opcji)
- Szczegółowe logowanie przebiegu gry

In [14]:
def run_clobber_ai(
    m=5,
    n=6,
    depth=3,
    heuristic='center',
    alpha_beta=True,
    black_heuristic=None,
    black_depth=None,
    white_heuristic=None,
    white_depth=None
):
    clobber = Clobber(n=n, m=m)
    board = clobber.board
    state = ClobberState(board, Pawn.BLACK)
    round_count = 0
    total_nodes = 0
    total_time = 0.0

    extended_mode = any([black_heuristic, white_heuristic, black_depth is not None, white_depth is not None])

    if extended_mode:
        # extended configuration
        if not all([black_heuristic, white_heuristic,
                    black_depth is not None, white_depth is not None]):
            raise ValueError(
                "Extended mode requires all of: black_heuristic, black_depth, white_heuristic, white_depth"
            )

        black_heuristic_func = HEURISTIC_MAP_EXTENDED.get(black_heuristic)
        white_heuristic_func = HEURISTIC_MAP_EXTENDED.get(white_heuristic)
        if black_heuristic_func is None or white_heuristic_func is None:
            raise ValueError(f"Invalid heuristic. Available: {list(HEURISTIC_MAP_EXTENDED.keys())}")

    else:
        # basic configuration
        if depth is None or heuristic is None:
            raise ValueError("Basic mode requires 'depth' and 'heuristic'")

        heuristic_func = HEURISTIC_MAP_EXTENDED.get(heuristic)
        if heuristic_func is None:
            raise ValueError(f"Invalid heuristic. Available: {list(HEURISTIC_MAP_EXTENDED.keys())}")

        black_heuristic_func = white_heuristic_func = heuristic_func
        black_depth = white_depth = depth

    use_alpha_beta = alpha_beta

    # MAIN LOOP
    stats = {'nodes': 0}
    while not state.is_terminal():
        current_player = state.current_player
        if current_player == Pawn.BLACK:
            curr_depth = black_depth
            curr_heuristic = black_heuristic_func
            player_name = "BLACK"
        else:
            curr_depth = white_depth
            curr_heuristic = white_heuristic_func
            player_name = "WHITE"

        move, value, nodes, elapsed = find_best_move(
            state, curr_depth, current_player, curr_heuristic, use_alpha_beta, stats
        )

        if move is None:
            print(f"{player_name} has no valid moves!")
            break

        print(f"Round {round_count + 1}, {player_name} move: {move}, value: {value:.2f}, nodes: {nodes}, time: {elapsed:.4f}s")
        state = state.make_move(move)
        round_count += 1
        total_nodes += nodes
        total_time += elapsed

    # end result
    print("Final board:")
    Clobber.print_board(state.board)

    winner = "NONE"
    if state.is_terminal():
        winner = "BLACK" if state.current_player == Pawn.WHITE else "WHITE"
    print(f"\nRounds: {round_count}, Winner: {winner}")
    print(f"Total nodes: {total_nodes}, Total time: {total_time:.4f}s")


## 7. Testowanie i wyniki
### Przykładowe uruchomienia

**Tryb podstawowy:**
```
run_clobber_ai(
    n=5,
    m=6,
    depth=2,
    heuristic='active',
    alpha_beta=True
)
```

**Tryb rozszerzony:**
```
run_clobber_ai(
    n=5,
    m=6,
    depth=3,
    black_depth=3,
    black_heuristic='adaptive1',
    white_depth=2,
    white_heuristic='accumulation',
    alpha_beta=True
)
```

In [15]:
run_clobber_ai(
    n=5,
    m=6,
    depth=2,
    heuristic='active',
    alpha_beta=True
)

Round 1, BLACK move: (0, 1, <Direction.DOWN: 2>), value: 0.00, nodes: 140, time: 0.0064s
Round 2, WHITE move: (2, 0, <Direction.DOWN: 2>), value: 0.00, nodes: 174, time: 0.0071s
Round 3, BLACK move: (0, 3, <Direction.DOWN: 2>), value: 0.00, nodes: 107, time: 0.0040s
Round 4, WHITE move: (0, 0, <Direction.DOWN: 2>), value: -1.00, nodes: 154, time: 0.0057s
Round 5, BLACK move: (1, 1, <Direction.LEFT: 4>), value: 0.00, nodes: 99, time: 0.0035s
Round 6, WHITE move: (0, 2, <Direction.DOWN: 2>), value: -1.00, nodes: 125, time: 0.0043s
Round 7, BLACK move: (1, 3, <Direction.LEFT: 4>), value: 0.00, nodes: 89, time: 0.0029s
Round 8, WHITE move: (0, 4, <Direction.DOWN: 2>), value: -1.00, nodes: 92, time: 0.0029s
Round 9, BLACK move: (1, 2, <Direction.DOWN: 2>), value: 0.00, nodes: 99, time: 0.0031s
Round 10, WHITE move: (2, 4, <Direction.DOWN: 2>), value: -1.00, nodes: 82, time: 0.0025s
Round 11, BLACK move: (2, 1, <Direction.DOWN: 2>), value: 0.00, nodes: 73, time: 0.0021s
Round 12, WHITE move:

In [16]:
run_clobber_ai(
    n=5,
    m=6,
    depth=3,
    black_depth=3,
    black_heuristic='first_center_then_aggressive',
    white_depth=2,
    white_heuristic='accumulation',
    alpha_beta=True
)

Round 1, BLACK move: (1, 2, <Direction.DOWN: 2>), value: 6.00, nodes: 6087, time: 0.1552s
Round 2, WHITE move: (4, 2, <Direction.RIGHT: 3>), value: -1.00, nodes: 597, time: 0.0245s
Round 3, BLACK move: (3, 0, <Direction.RIGHT: 3>), value: 12.00, nodes: 3141, time: 0.0748s
Round 4, WHITE move: (0, 0, <Direction.DOWN: 2>), value: -1.00, nodes: 118, time: 0.0047s
Round 5, BLACK move: (0, 1, <Direction.DOWN: 2>), value: 15.00, nodes: 1616, time: 0.0352s
Round 6, WHITE move: (0, 2, <Direction.RIGHT: 3>), value: -1.00, nodes: 92, time: 0.0033s
Round 7, BLACK move: (1, 4, <Direction.LEFT: 4>), value: 20.00, nodes: 757, time: 0.0160s
Round 8, WHITE move: (2, 4, <Direction.LEFT: 4>), value: -1.00, nodes: 126, time: 0.0042s
Round 9, BLACK move: (3, 4, <Direction.LEFT: 4>), value: 18.00, nodes: 440, time: 0.0086s
Round 10, WHITE move: (2, 3, <Direction.DOWN: 2>), value: -1.00, nodes: 106, time: 0.0032s
Round 11, BLACK move: (5, 4, <Direction.UP: 1>), value: 17.00, nodes: 316, time: 0.0057s
Round 

### Wnioski z testowania
1. **Wpływ głębokości przeszukiwania:**
    - Zwiększenie głębokości poprawia jakość ruchów
    - Kosztem jest wzrost czasu obliczeń i liczby odwiedzonych węzłów
2. **Skuteczność cięć alfa-beta:**
    - Znacząca redukcja liczby odwiedzanych wierzchołków
    - Przyspieszenie czasu wykonania algorytmu
    - Możliwość zastosowania większego poziomu głębokości (dzieki skróconemu czasowi)
3. **Nie zawsze wygrywa rozpoczynający grę** - nawet jeśli gra odbywa się z perspektywy jednego agenta, który dla obu stron wykorzystuje te same heurystyki i głębokości, nie zawsze wygrywa gracz czarny, który z racji, że zaczyna - ma większą przewagę. Wynika to najprawdopodobniej z niedoskonałości używanych heurystyk i niskiego poziomu głębokości. Wspólnie dwa te czynniki mogą powodować wykonywanie ruchów prowadzących w większej perspektywie do przegranej.

## 8. Podsumowanie i wnioski
### Osiągnięte cele:
1. Poprawna implementacja gry Clobber
2. Zrealizowanie algorytmów Minimax i Alpha-Beta
3. Implementacja różnych heurystyk oceny stanu gry (również adaptacyjnych)
4. Wykonanie zarówno trybu podstawowego, jak i rozszerzonego

### Napotkane problemy:
1. **Nieznajomość gry** - w związku z tym, że wcześniej nie miałem do czynienia z grą Clobber, na początku musiałem zrozumieć jej zasady. Jednak samo zrozumienie zasad to niestety za mało, gdyż zadanie wymagało utworzenia odpowiednich heurystyk odzwierciedlających stan gry - wymagała to większego zgłębienia strategii w grze (co również nie było łatwym zadaniem biorąc pod uwagę, że ciężko znaleźć nawet jakiekolwiek źródła na internecie, które by cokolwiek o tej grze i strategii z nią związanych wspominały.
   
2. **Tworzenie heurystyk** - nawet po zapoznaniu sie z grą, ze względu na jej specyfikę ciężko było wymyślić jakiekolwiek sensowne heurystyki. Chwilowe zapoznanie się z grą to zdecydowanie za mało na wymyślenie czegoś sensownego, tym bardziej, że gra ma bardzo nietypowe zasady, powodujące np. że zawsze liczba pionków obu graczy jest sobie równa lub różni się o 1, przez co dużo potencjalnych heurystyk związanych z ilością pionów, jakie możnaby zastosować w innych, podobnych grach tutaj się nie nadawały.
   
3. **Poprawność algorytmu** - algorytm minimax oraz cięcie alfa-beta mimo tego, że nie są algorytmami mocno skomplikowanymi, z racji implementacji ich po raz pierwszy, przysporzyły sporo problemów, zarówno natury omyłkowej (małych błędów w kodzie, literówek), jak i poważniejszych, wynikających z nieprawidłowego zrozumienia działania algorytmu, które zmuszały do poważnych zmian w implementacji.


## 9. Wykorzystane biblioteki
1. **enum** - biblioteka pozwalająca na implementację klasy enumeratora w pythonie. W moim przypadku wykorzystana celem utworzenia klas takich jak Direction, Pawn, czy GameStatus

2. **functools** - biblioteka z funkcjami wyższego rzędu. Wykorzystałem z niej funkcję lru_cache, dzięki której mogłem cache'ować dane w funkcji generującej macierz wag (do heurystyki), która była często wywoływana z tymi samymi parametrami.

3. **typing** - biblioteka zawierająca typy danych w pythonie. Używana celem jawnego podawania typów (zwiększenie estetyki i przejrzystości kodu) a także tworzenia własnych aliasów, np. Board

4. **time** - biblioteka zawierająca funkcje związane z czasem. Użyta do logowania czasu trwania poszczególnych etapów liczenia w algorytmach.

## 10. Źródła
1. https://www.youtube.com/watch?v=zahxNZYvvj8 - film przedstawiający rozgrywkę w Clobbera
2. https://www.iggamecenter.com/en/rules/clobber - podstawowe zasady Clobbera